Preparations

In [1]:
# # check API key
# import os

# api_key = os.environ.get("OPENAI_API_KEY")
# if api_key:
#     print("API key is set")
# else:
#     print("API key not found in environment variables")

In [2]:
from mira.openai import OpenAIClient
from mira.sources.sympy_ode.llm_util import execute_template_model_from_sympy_odes, image_file_to_odes_str
client = OpenAIClient()

In [3]:
# Extract ODEs from images
import os
from IPython.display import Image, display

image_folder = './images'  

image_files = [f for f in os.listdir(image_folder) if not f.startswith('.')] # images that do not start with a dot in their file name

print(f"Found {len(image_files)} files to process:")
for img_file in image_files:
    print(f"  - {img_file}")

for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    print(f"\nProcessing: {image_file}")
    #display(Image(image_path))

Found 24 files to process:
  - BIOMD0000000972.png
  - BIOMD0000000964.png
  - BIOMD0000000970.png
  - BIOMD0000000958.png
  - BIOMD0000000971.png
  - BIOMD0000000974.png
  - BIOMD0000000960.png
  - BIOMD0000000976.png
  - BIOMD0000000962.png
  - SAPHIRE.png
  - BIOMD0000000963.png
  - BIOMD0000000977.png
  - 2024_dec_epi_1_model_C.png
  - 2024_dec_epi_1_model_B.png
  - odes_to_mira_SEVITHR.png
  - 2024_dec_epi_1_model_A.png
  - BIOMD0000000991.png
  - BIOMD0000000984.png
  - BIOMD0000000979.png
  - BIOMD0000000978.png
  - BIOMD0000000983.png
  - BIOMD0000000955.png
  - BIOMD0000000957.png
  - BIOMD0000000956.png

Processing: BIOMD0000000972.png

Processing: BIOMD0000000964.png

Processing: BIOMD0000000970.png

Processing: BIOMD0000000958.png

Processing: BIOMD0000000971.png

Processing: BIOMD0000000974.png

Processing: BIOMD0000000960.png

Processing: BIOMD0000000976.png

Processing: BIOMD0000000962.png

Processing: SAPHIRE.png

Processing: BIOMD0000000963.png

Processing: BIOMD000000

In [4]:
#for one image only:
# ode_str = image_file_to_odes_str(image_path, client=client)
# print(ode_str)

# Note that the LLM sometimes misses to define one of the parameters. When this happens, a human-in-the-loop can quickly edit the output:

# Cleaned version of string above, uncomment to use it
# ode_str = """import sympy

# # Define time variable
# t = sympy.symbols("t")

# # Define the time-dependent variables
# S, E, I, T, R, V, H = sympy.symbols("S E I T R V H", cls=sympy.Function)

# # Define the parameters
# lambda_, epsilon_v, p_T, delta, eta, kappa, mu, sigma = sympy.symbols("lambda_ epsilon_v p_T delta eta kappa mu sigma")

# odes = [
#    sympy.Eq(S(t).diff(t), - lambda_ * S(t)),
#    sympy.Eq(E(t).diff(t), lambda_ * S(t) + (1 - epsilon_v) * lambda_ * V(t) - sigma * E(t)),
#    sympy.Eq(I(t).diff(t), (1 - p_T) * sigma * E(t) - (delta + eta) * I(t)),
#    sympy.Eq(T(t).diff(t), p_T * sigma * E(t) - delta * T(t)),
#    sympy.Eq(R(t).diff(t), delta * I(t) + delta * T(t) + kappa * H(t)),
#    sympy.Eq(V(t).diff(t), - (1 - epsilon_v) * lambda_ * V(t)),
#    sympy.Eq(H(t).diff(t), eta * I(t) - (kappa + mu) * H(t))
#     ]"""

# print(ode_str)

In [5]:
# for multiple images:

all_results = []
failed_extractions = []

for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    print(f"\n{'='*60}")
    print(f"Processing: {image_file}")
    print(f"{'='*60}")
    
    try:        
        ode_str = image_file_to_odes_str(image_path, client)
        print("Extracted odes:")
        print(ode_str[:500])  
        
        template_model = execute_template_model_from_sympy_odes(
            ode_str=ode_str,
            attempt_grounding=True,
            client=client,
            use_multi_agent=True,
            max_correction_iterations=3
        )
        
        concepts_map = template_model.get_concepts_map()
        
        result = {
            'image_file': image_file,
            'template_model': template_model,
            'status': 'success'
        }
        all_results.append(result)
        
        print(f"Successfully extracted from {image_file}")
        print(f"  - Templates: {len(template_model.templates)}")
        print(f"  - Concepts: {len(concepts_map)}")
        print(f"  - Parameters: {len(template_model.parameters)}")
        
    except Exception as e:
        print(f"✗ Failed to process {image_file}: {str(e)}")
        failed_extractions.append({
            'image_file': image_file,
            'error': str(e),
            'status': 'failed'
        })

print(f"\n{'='*60}")
print("PROCESSING SUMMARY")
print(f"{'='*60}")
print(f"Total files: {len(image_files)}")
print(f"Successful extractions: {len(all_results)}")
print(f"Failed extractions: {len(failed_extractions)}")

if failed_extractions:
    print(f"\nFailed files:")
    for failure in failed_extractions:
        print(f"  - {failure['image_file']}: {failure['error']}")
        



Processing: BIOMD0000000972.png


INFO: [2025-08-28 09:45:27] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO: [2025-08-28 09:45:27] openai._base_client - Retrying request to /chat/completions in 0.436000 seconds
INFO: [2025-08-28 09:45:28] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO: [2025-08-28 09:45:28] openai._base_client - Retrying request to /chat/completions in 0.979354 seconds
INFO: [2025-08-28 09:45:30] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"


✗ Failed to process BIOMD0000000972.png: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

Processing: BIOMD0000000964.png


INFO: [2025-08-28 09:45:32] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO: [2025-08-28 09:45:32] openai._base_client - Retrying request to /chat/completions in 0.392887 seconds
INFO: [2025-08-28 09:45:33] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO: [2025-08-28 09:45:33] openai._base_client - Retrying request to /chat/completions in 0.762555 seconds
INFO: [2025-08-28 09:45:34] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO: [2025-08-28 09:45:34] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO: [2025-08-28 09:45:34] openai._base_client - Retrying request to /chat/completions in 0.403731 seconds


✗ Failed to process BIOMD0000000964.png: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

Processing: BIOMD0000000970.png


...

✗ Failed to process 2024_dec_epi_1_model_B.png: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

Processing: odes_to_mira_SEVITHR.png


INFO: [2025-08-28 09:45:56] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO: [2025-08-28 09:45:56] openai._base_client - Retrying request to /chat/completions in 0.967208 seconds
INFO: [2025-08-28 09:45:57] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO: [2025-08-28 09:45:57] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO: [2025-08-28 09:45:57] openai._base_client - Retrying request to /chat/completions in 0.430820 seconds


✗ Failed to process odes_to_mira_SEVITHR.png: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

Processing: 2024_dec_epi_1_model_A.png


INFO: [2025-08-28 09:45:58] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO: [2025-08-28 09:45:58] openai._base_client - Retrying request to /chat/completions in 0.791499 seconds


KeyboardInterrupt: 

In [ ]:
# for one image only:
# Now get a template model from the extracted ODE str and send the ODE system in for attempting to ground each compartment
#from mira.sources.sympy_ode.llm_util import execute_template_model_from_sympy_odes

#tm = execute_template_model_from_sympy_odes(ode_str=ode_str, attempt_grounding=True, client=client)

In [ ]:
# for one image:
# Check the concepts
# print('concept name\tidentifiers\tcontext')
# for concept in tm.get_concepts_map().values():
#     print(f'{concept.name}\t{concept.identifiers}\t{concept.context}')

# for all images:
for result in all_results:
    tm = result['template_model']
    image_file = result['image_file']
    
    print(f"\n=== Concepts from {image_file} ===")
    concepts_map = tm.get_concepts_map()
    
    if concepts_map:
        print('concept name\tidentifiers\tcontext')
        for concept in concepts_map.values():
            print(f'{concept.name}\t{concept.identifiers}\t{concept.context}')
    else:
        print("No concepts found")

In [ ]:
# from mira.modeling import Model
# from mira.modeling.ode import OdeModel
# om = OdeModel(Model(tm), initialized=True)
# om.get_interpretable_kinetics()

Above: let's save the results here: *mira/notebooks/equation extraction development/extraction error check/string mismatch check/extracted_eqs_all versions_EXCEL.xlsx*

In [ ]:
import pandas as pd
from pathlib import Path

let's add a variable "version" to see, which version of the AI architecture are we using at the moment:

In [ ]:
version = '002' # this is the 2-step iterative architecture but change the number to a different one

In [ ]:
def save_all_ode_results_with_strings(all_results, failed_extractions, version="002"):
    """Save actual ODE strings for all images"""
    
    excel_path = './equation extraction development/extraction error check/string mismatch check/extracted_eqs_all versions_EXCEL.xlsx'
    version_col = f'version{version}'
    
    try:
        df = pd.read_excel(excel_path, engine='openpyxl')
    except FileNotFoundError:
        df = pd.DataFrame(columns=['model', version_col]) 
    
    # Process successful results - extract ONLY the odes part
    for result in all_results:
        model_name = Path(result['image_file']).stem
        full_ode_str = result.get('ode_str', '')
        
        # Extract only the odes = [...] part
        if full_ode_str:
            start = full_ode_str.find('odes = [')
            end = full_ode_str.find(']', start) + 1
            
            if start != -1 and end != 0:
                ode_str_clean = full_ode_str[start:end]
            else:
                ode_str_clean = "No 'odes = [...]' pattern found"
        else:
            ode_str_clean = 'SUCCESS - No ODE string stored'
        
        # Update or add row
        if 'model' in df.columns and not df.empty:
            model_exists = df['model'] == model_name
            if model_exists.any():
                df.loc[model_exists, version_col] = ode_str_clean
            else:
                new_row = pd.DataFrame({'model': [model_name], version_col: [ode_str_clean]})
                df = pd.concat([df, new_row], ignore_index=True)
        else:
            df = pd.DataFrame({'model': [model_name], version_col: [ode_str_clean]})
    
    # Process failures
    for failure in failed_extractions:
        model_name = Path(failure['image_file']).stem
        error_msg = f"FAILED: {failure['error'][:100]}..."
        
        if 'model' in df.columns and not df.empty:
            model_exists = df['model'] == model_name
            if model_exists.any():
                df.loc[model_exists, version_col] = error_msg
            else:
                new_row = pd.DataFrame({'model': [model_name], version_col: [error_msg]})
                df = pd.concat([df, new_row], ignore_index=True)
        else:
            df = pd.DataFrame({'model': [model_name], version_col: [error_msg]})
    
    df.to_excel(excel_path, index=False, engine='openpyxl')
    print(f"Saved clean ODE strings for {len(all_results)} successful and {len(failed_extractions)} failed extractions")

In [ ]:
# def save_ode_results(ode_str, input_filename):
#     """
#     Save ODE extraction results to Excel file.
    
#     Parameters:
#     -----------
#     ode_str : str
#         The extracted ODEs from MIRA
#     input_filename : str
#         The input file name (e.g., 'model_name.png')
#     """

#     excel_path = './equation extraction development/extraction error check/string mismatch check/extracted_eqs_all versions_EXCEL.xlsx'
#     model_name = Path(input_filename).stem

#     try:
#         df = pd.read_excel(excel_path, engine='openpyxl')
#     except FileNotFoundError:
#         df = pd.DataFrame(columns=['model', 'version{version}']) 

#     if 'model' in df.columns:
#         model_exists = df['model'] == model_name

#         if model_exists.any():
#             df.loc[model_exists, 'version002'] = ode_str
#         else:
#             new_row = pd.DataFrame({
#                 'model': [model_name],
#                 'version002': [ode_str]
#             })
#             df = pd.concat([df, new_row], ignore_index=True)
#     else:
#         df['model'] = model_name
#         df['version{version}'] = ode_str

#     df.to_excel(excel_path, index=False, engine='openpyxl')
#     print(f"Saved equations for model '{model_name}'")

In [ ]:
# for one image only:
# start = ode_str.find('odes = [')
# end = ode_str.find(']', start) + 1
# ode_str_clean = ode_str[start:end]

In [ ]:
#for one image:
# save_ode_results(ode_str_clean, image_path)

# for all images:
save_all_ode_results_with_strings(all_results, failed_extractions, version="002")

Saved clean ODE strings for 0 successful and 24 failed extractions


In [ ]:
def create_tsv_from_excel(version='002'):
    
    excel_path = './equation extraction development/extraction error check/string mismatch check/extracted_eqs_all versions_EXCEL.xlsx'
    df = pd.read_excel(excel_path, engine='openpyxl')
    
    tsv_df = pd.DataFrame({
        'model': df['model'],
        'extracted_eqs': df[f'version{version}']
    })
    
    tsv_df = tsv_df.dropna(subset=['extracted_eqs'])
    
    tsv_path = f'./equation extraction development/extraction error check/string mismatch check/extracted_eqs_VERSION{version}.tsv'
    tsv_df.to_csv(tsv_path, sep='\t', index=False)
    print(f"TSV file saved: {tsv_path}")

create_tsv_from_excel(version)

TSV file saved: ./equation extraction development/extraction error check/string mismatch check/extracted_eqs_VERSION002.tsv
